<a href="https://colab.research.google.com/github/shivang2607/Recommendation-System-Machine-Learning-/blob/main/Manga%20Recommendation%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Downloading dataset to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [ ]:
%cd /content/drive/MyDrive/Kaggle/

/content/drive/MyDrive/Kaggle


In [ ]:
!kaggle datasets download -d andreuvallhernndez/myanimelist --unzip


 67% 17.0M/25.5M [00:00<00:00, 64.2MB/s]
100% 25.5M/25.5M [00:00<00:00, 80.4MB/s]


In [ ]:
%ls

anime.csv  kaggle.json  manga.csv


#Data Cleaning and Preprocessing


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors, KDTree
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import joblib



In [ ]:
%cd /content/drive/MyDrive
# Current Location is Kaggle Folder, use cd command to change it.
manga = pd.read_csv("Kaggle/manga.csv")
manga.head()
manga.info()

/content/drive/MyDrive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67273 entries, 0 to 67272
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   manga_id           67273 non-null  int64  
 1   title              67273 non-null  object 
 2   type               67273 non-null  object 
 3   score              21910 non-null  float64
 4   scored_by          67273 non-null  int64  
 5   status             67273 non-null  object 
 6   volumes            50494 non-null  float64
 7   chapters           47095 non-null  float64
 8   start_date         62949 non-null  object 
 9   end_date           51721 non-null  object 
 10  members            67273 non-null  int64  
 11  favorites          67273 non-null  int64  
 12  sfw                67273 non-null  bool   
 13  approved           67273 non-null  bool   
 14  created_at_before  67273 non-null  object 
 15  updated_at         58991 non-null  object 
 16 

In [ ]:
manga = manga[['manga_id','title', 'type', 'score', 'status', 'volumes','main_picture','url', 'chapters', 'start_date','genres', 'themes','demographics','sfw','authors','title_english','title_japanese']]
manga.head()

,manga_id,title,type,score,status,volumes,main_picture,url,chapters,start_date,genres,themes,demographics,sfw,authors,title_english,title_japanese
0,2,Berserk,manga,9.45,currently_publishing,NaN,https://cdn.myanimelist.net/images/manga/1/157...,https://myanimelist.net/manga/2/Berserk,NaN,1989-08-25,"['Action', 'Adventure', 'Award Winning', 'Dram...","['Gore', 'Military', 'Mythology', 'Psychologic...",['Seinen'],True,"[{'id': 1868, 'first_name': 'Kentarou', 'last_...",Berserk,ベルセルク
1,13,One Piece,manga,9.20,currently_publishing,NaN,https://cdn.myanimelist.net/images/manga/2/253...,https://myanimelist.net/manga/13/One_Piece,NaN,1997-07-22,"['Action', 'Adventure', 'Fantasy']",[],['Shounen'],True,"[{'id': 1881, 'first_name': 'Eiichiro', 'last_...",One Piece,ONE PIECE
2,25,Fullmetal Alchemist,manga,9.05,finished,27.0,https://cdn.myanimelist.net/images/manga/3/243...,https://myanimelist.net/manga/25/Fullmetal_Alc...,116.0,2001-07-12,"['Action', 'Adventure', 'Award Winning', 'Dram...",['Military'],['Shounen'],True,"[{'id': 1874, 'first_name': 'Hiromu', 'last_na...",Fullmetal Alchemist,鋼の錬金術師
3,1706,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,manga,9.27,finished,24.0,https://cdn.myanimelist.net/images/manga/3/179...,https://myanimelist.net/manga/1706/JoJo_no_Kim...,96.0,2004-01-19,"['Action', 'Adventure', 'Horror', 'Mystery', '...",['Historical'],"['Seinen', 'Shounen']",True,"[{'id': 2619, 'first_name': 'Hirohiko', 'last_...",NaN,ジョジョの奇妙な冒険 Part7 STEEL BALL RUN
4,4632,Oyasumi Punpun,manga,9.03,finished,13.0,https://cdn.myanimelist.net/images/manga/3/164...,https://myanimelist.net/manga/4632/Oyasumi_Punpun,147.0,2007-03-15,"['Drama', 'Slice of Life']",['Psychological'],['Seinen'],True,"[{'id': 2836, 'first_name': 'Inio', 'last_name...",Goodnight Punpun,おやすみプンプン


In [ ]:
manga['type'].value_counts()
mangac = manga.copy()
mangac=mangac[mangac['authors'].str.len()>2]
mangac.reset_index(inplace=True)
mangac.drop('index',1, inplace = True)



<ipython-input-18-e17ddd194d69>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  mangac.drop('index',1, inplace = True)


In [ ]:
mangac = mangac[(mangac['score']>6.35)]
mangac.to_csv('manga_filtered')


In [ ]:

mangac['genres']=mangac['genres'].apply(eval)
mangac['themes']=mangac['themes'].apply(eval)
mangac['demographics']=mangac['demographics'].apply(eval)


In [ ]:
import ast
mangac.head(5)

mangac['authors']=mangac['authors'].apply(lambda x: ast.literal_eval(x))
mangac['authors_id']=mangac['authors'].apply(lambda x: x[0]['id'])



In [ ]:
mangac['authors_id'].value_counts()
mangac.info()
mangac['authors_id'].fillna(0, inplace=True)
mangac['authors_id']=mangac['authors_id'].astype(str)
# mangac['themes'].fillna("")
top_100_authors = mangac['authors_id'].value_counts()[:100].index.to_list()
mangac['authors_id']=mangac['authors_id'].apply(lambda x: x if x in top_100_authors else '0')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59750 entries, 0 to 59749
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   manga_id        59750 non-null  int64  
 1   title           59750 non-null  object 
 2   type            59750 non-null  object 
 3   score           21862 non-null  float64
 4   status          59750 non-null  object 
 5   volumes         45638 non-null  float64
 6   main_picture    59638 non-null  object 
 7   url             59750 non-null  object 
 8   chapters        42602 non-null  float64
 9   start_date      57937 non-null  object 
 10  genres          59750 non-null  object 
 11  themes          59750 non-null  object 
 12  demographics    59750 non-null  object 
 13  sfw             59750 non-null  bool   
 14  authors         59750 non-null  object 
 15  title_english   15775 non-null  object 
 16  title_japanese  58202 non-null  object 
 17  authors_id      59750 non-null 

In [ ]:
mangac['text_input']=mangac['themes']+mangac['genres']+mangac['demographics']
mangac.to_parquet("animovix-models and files/manga.parquet", compression='GZIP')

In [ ]:

mangac

,manga_id,title,type,score,status,volumes,main_picture,url,chapters,start_date,genres,themes,demographics,sfw,authors,title_english,title_japanese,authors_id,text_input
0,2,Berserk,manga,9.45,currently_publishing,NaN,https://cdn.myanimelist.net/images/manga/1/157...,https://myanimelist.net/manga/2/Berserk,NaN,1989-08-25,"[Action, Adventure, Award Winning, Drama, Fant...","[Gore, Military, Mythology, Psychological]",[Seinen],True,"[{'id': 1868, 'first_name': 'Kentarou', 'last_...",Berserk,ベルセルク,0,"[Gore, Military, Mythology, Psychological, Act..."
1,13,One Piece,manga,9.20,currently_publishing,NaN,https://cdn.myanimelist.net/images/manga/2/253...,https://myanimelist.net/manga/13/One_Piece,NaN,1997-07-22,"[Action, Adventure, Fantasy]",[],[Shounen],True,"[{'id': 1881, 'first_name': 'Eiichiro', 'last_...",One Piece,ONE PIECE,0,"[Action, Adventure, Fantasy, Shounen]"
2,25,Fullmetal Alchemist,manga,9.05,finished,27.0,https://cdn.myanimelist.net/images/manga/3/243...,https://myanimelist.net/manga/25/Fullmetal_Alc...,116.0,2001-07-12,"[Action, Adventure, Award Winning, Drama, Fant...",[Military],[Shounen],True,"[{'id': 1874, 'first_name': 'Hiromu', 'last_na...",Fullmetal Alchemist,鋼の錬金術師,0,"[Military, Action, Adventure, Award Winning, D..."
3,1706,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,manga,9.27,finished,24.0,https://cdn.myanimelist.net/images/manga/3/179...,https://myanimelist.net/manga/1706/JoJo_no_Kim...,96.0,2004-01-19,"[Action, Adventure, Horror, Mystery, Supernatu...",[Historical],"[Seinen, Shounen]",True,"[{'id': 2619, 'first_name': 'Hirohiko', 'last_...",NaN,ジョジョの奇妙な冒険 Part7 STEEL BALL RUN,0,"[Historical, Action, Adventure, Horror, Myster..."
4,4632,Oyasumi Punpun,manga,9.03,finished,13.0,https://cdn.myanimelist.net/images/manga/3/164...,https://myanimelist.net/manga/4632/Oyasumi_Punpun,147.0,2007-03-15,"[Drama, Slice of Life]",[Psychological],[Seinen],True,"[{'id': 2836, 'first_name': 'Inio', 'last_name...",Goodnight Punpun,おやすみプンプン,0,"[Psychological, Drama, Slice of Life, Seinen]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59745,132714,Pokemon wo Tsukutta Otoko: Tajiri Satoshi,manga,NaN,finished,1.0,https://cdn.myanimelist.net/images/manga/3/239...,https://myanimelist.net/manga/132714/Pokemon_w...,8.0,2018-05-16,[],[Historical],[],True,"[{'id': 53367, 'first_name': 'Ken', 'last_name...",NaN,ポケモンをつくった男 田尻智,0,[Historical]
59746,135380,Kuroudo,manga,NaN,finished,10.0,https://cdn.myanimelist.net/images/manga/1/244...,https://myanimelist.net/manga/135380/Kuroudo,89.0,2006-01-01,[],[],[Seinen],True,"[{'id': 4526, 'first_name': 'Akira', 'last_nam...",NaN,蔵人,0,[Seinen]
59747,137118,Manga de Wakaru Linux: Sys Kankei Joshi,manga,NaN,currently_publishing,NaN,https://cdn.myanimelist.net/images/manga/2/245...,https://myanimelist.net/manga/137118/Manga_de_...,NaN,2011-08-08,[Comedy],[],[],True,"[{'id': 54409, 'first_name': '', 'last_name': ...",System Admin Girl★,まんがでわかるLinux シス管系女子,0,[Comedy]
59748,144533,Watashi to Otto to Otto no Kareshi,manga,NaN,currently_publishing,NaN,https://cdn.myanimelist.net/images/manga/2/256...,https://myanimelist.net/manga/144533/Watashi_t...,NaN,2020-12-25,"[Drama, Romance]",[],[],True,"[{'id': 45352, 'first_name': 'Ayano', 'last_na...","Me, My Husband & My Husband's Boyfriend",私と夫と夫の彼氏,0,"[Drama, Romance]"


In [ ]:
CountVectorizer(analyzer=set).fit_transform(mangac['text_input']).toarray().shape

(59750, 78)

In [ ]:
num_pipeline = Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
                ('ordinal_encoder', OrdinalEncoder()),
                ('simple_imputer', SimpleImputer(strategy='most_frequent')),
                ('categorical_encoder', OneHotEncoder()),
])



list_pipeline = Pipeline([
                ('count_vectoriser', CountVectorizer(analyzer=set)),

])
# animeb.head()




preprocess_pipeline = ColumnTransformer([
                        ('num_columns',num_pipeline, ['score']),
                        ('cat_columns', cat_pipeline, ['type', 'sfw', 'authors_id']),
                        ('list_columns', list_pipeline, 'text_input'),
])


train = preprocess_pipeline.fit_transform(mangac)
mangac

,manga_id,title,type,score,status,volumes,main_picture,url,chapters,start_date,genres,themes,demographics,sfw,authors,title_english,title_japanese,authors_id,text_input
0,2,Berserk,manga,9.45,currently_publishing,NaN,https://cdn.myanimelist.net/images/manga/1/157...,https://myanimelist.net/manga/2/Berserk,NaN,1989-08-25,"[Action, Adventure, Award Winning, Drama, Fant...","[Gore, Military, Mythology, Psychological]",[Seinen],True,"[{'id': 1868, 'first_name': 'Kentarou', 'last_...",Berserk,ベルセルク,0,"[Gore, Military, Mythology, Psychological, Act..."
1,13,One Piece,manga,9.20,currently_publishing,NaN,https://cdn.myanimelist.net/images/manga/2/253...,https://myanimelist.net/manga/13/One_Piece,NaN,1997-07-22,"[Action, Adventure, Fantasy]",[],[Shounen],True,"[{'id': 1881, 'first_name': 'Eiichiro', 'last_...",One Piece,ONE PIECE,0,"[Action, Adventure, Fantasy, Shounen]"
2,25,Fullmetal Alchemist,manga,9.05,finished,27.0,https://cdn.myanimelist.net/images/manga/3/243...,https://myanimelist.net/manga/25/Fullmetal_Alc...,116.0,2001-07-12,"[Action, Adventure, Award Winning, Drama, Fant...",[Military],[Shounen],True,"[{'id': 1874, 'first_name': 'Hiromu', 'last_na...",Fullmetal Alchemist,鋼の錬金術師,0,"[Military, Action, Adventure, Award Winning, D..."
3,1706,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,manga,9.27,finished,24.0,https://cdn.myanimelist.net/images/manga/3/179...,https://myanimelist.net/manga/1706/JoJo_no_Kim...,96.0,2004-01-19,"[Action, Adventure, Horror, Mystery, Supernatu...",[Historical],"[Seinen, Shounen]",True,"[{'id': 2619, 'first_name': 'Hirohiko', 'last_...",NaN,ジョジョの奇妙な冒険 Part7 STEEL BALL RUN,0,"[Historical, Action, Adventure, Horror, Myster..."
4,4632,Oyasumi Punpun,manga,9.03,finished,13.0,https://cdn.myanimelist.net/images/manga/3/164...,https://myanimelist.net/manga/4632/Oyasumi_Punpun,147.0,2007-03-15,"[Drama, Slice of Life]",[Psychological],[Seinen],True,"[{'id': 2836, 'first_name': 'Inio', 'last_name...",Goodnight Punpun,おやすみプンプン,0,"[Psychological, Drama, Slice of Life, Seinen]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59745,132714,Pokemon wo Tsukutta Otoko: Tajiri Satoshi,manga,NaN,finished,1.0,https://cdn.myanimelist.net/images/manga/3/239...,https://myanimelist.net/manga/132714/Pokemon_w...,8.0,2018-05-16,[],[Historical],[],True,"[{'id': 53367, 'first_name': 'Ken', 'last_name...",NaN,ポケモンをつくった男 田尻智,0,[Historical]
59746,135380,Kuroudo,manga,NaN,finished,10.0,https://cdn.myanimelist.net/images/manga/1/244...,https://myanimelist.net/manga/135380/Kuroudo,89.0,2006-01-01,[],[],[Seinen],True,"[{'id': 4526, 'first_name': 'Akira', 'last_nam...",NaN,蔵人,0,[Seinen]
59747,137118,Manga de Wakaru Linux: Sys Kankei Joshi,manga,NaN,currently_publishing,NaN,https://cdn.myanimelist.net/images/manga/2/245...,https://myanimelist.net/manga/137118/Manga_de_...,NaN,2011-08-08,[Comedy],[],[],True,"[{'id': 54409, 'first_name': '', 'last_name': ...",System Admin Girl★,まんがでわかるLinux シス管系女子,0,[Comedy]
59748,144533,Watashi to Otto to Otto no Kareshi,manga,NaN,currently_publishing,NaN,https://cdn.myanimelist.net/images/manga/2/256...,https://myanimelist.net/manga/144533/Watashi_t...,NaN,2020-12-25,"[Drama, Romance]",[],[],True,"[{'id': 45352, 'first_name': 'Ayano', 'last_na...","Me, My Husband & My Husband's Boyfriend",私と夫と夫の彼氏,0,"[Drama, Romance]"


In [ ]:
print(train.shape)
K_model_cos = NearestNeighbors(n_neighbors=100, metric='cosine')
K_model_cos.fit(train)
joblib.dump(K_model_cos, 'animovix-models and files/K_model_manga.sav')
joblib.dump(preprocess_pipeline, 'animovix-models and files/preprocess_pipeline_manga.sav')


(59750, 189)


['animovix-models and files/preprocess_pipeline_manga.sav']

In [ ]:
mangac[mangac['title'].str.lower().str.contains('naruto')][:1]

,manga_id,title,type,score,status,volumes,main_picture,url,chapters,start_date,genres,themes,demographics,sfw,authors,title_english,title_japanese,authors_id,text_input
194,11,Naruto,manga,8.06,finished,72.0,https://cdn.myanimelist.net/images/manga/3/249...,https://myanimelist.net/manga/11/Naruto,700.0,1999-09-21,"[Action, Adventure, Fantasy]",[Martial Arts],[Shounen],True,"[{'id': 1879, 'first_name': 'Masashi', 'last_n...",Naruto,NARUTO―ナルト―,0,"[Martial Arts, Action, Adventure, Fantasy, Sho..."


In [ ]:
#change the anime name in below line it will search in the dataframe with the name and convert that row into vector and
# show nearest neighbors for that Anime (firstly recommendations from cosine metrixs is shown then recomm. with l2 metrics)
train=joblib.load('animovix-models and files/manga_train.sav')
K_model_cos=joblib.load('animovix-models and files/K_model_manga.sav')
sample = preprocess_pipeline.transform(mangac[mangac['title'].str.lower().str.contains('naruto')][:1])
sample
distances_cos, indices_cos = K_model_cos.kneighbors(sample)
# distances_cos, indices_cos =
pred = mangac['title'].loc[indices_cos.squeeze()[0:]].tolist()
pred

['Naruto',
 'Magi: Sinbad no Bouken',
 'Dragon Quest: Dai no Daibouken',
 'Hunter x Hunter',
 'Dungeon ni Deai wo Motomeru no wa Machigatteiru Darou ka Gaiden: Sword Oratoria',
 'Rain',
 'Naruto Shinden Series',
 'Tales of Symphonia',
 'JoJo no Kimyou na Bouken Part 6: Stone Ocean',
 'Saiyuuki Gaiden',
 'Zelda no Densetsu: Twilight Princess',
 'Kenji',
 'Kingdom Hearts II',
 'Black Cat',
 'Tales of Symphonia: Extra Load',
 'Claymore',
 'One Piece',
 'One Piece: Episode A',
 'Hibiki no Mahou',
 'Bleach',
 'Gensou Suikoden III: Unmei no Keishousha',
 '+Anima',
 'Magi',
 'Kimetsu no Yaiba',
 'JoJo no Kimyou na Bouken Part 5: Ougon no Kaze',
 'Hanma Baki',
 'One Piece: Strong World',
 'Noragami',
 'JoJo no Kimyou na Bouken Part 3: Stardust Crusaders',
 'The Legend of the Northern Blade',
 'Soul Eater',
 'Ousama Ranking',
 'Saiyuuki',
 'Kingdom Hearts: 358/2 Days',
 'Jujutsu Kaisen',
 'Senyuu.',
 'Sousou no Frieren',
 'Hitsugikatsugi no Kuro.: Kaichu Tabi no Wa',
 'Helck',
 'Fairy Tail Zerø

In [ ]:
import json
mangac=pd.read_parquet('animovix-models and files/manga.parquet')
pipeline = joblib.load('animovix-models and files/preprocess_pipeline_manga.sav')
K_model_cos=joblib.load('animovix-models and files/K_model_manga.sav')

input='death note'
input = (input.lower()).split('|')              # corresponding vector representations will be found coresponding to
                                                    # which Final Recommendations will be given as output.
samples=[]
for name in input:
    samples.append((pipeline.transform(mangac[mangac['title'].apply(lambda x:name ==  x.lower())][:1])).toarray())
print(samples)
sample_mean = [sum(sub_list) / len(sub_list) for sub_list in zip(*samples)] # finding mean of all nodes and
distances_cos_comb, indices_cos_comb = K_model_cos.kneighbors(sample_mean)        # treating it as a midpoint
results = mangac.iloc[indices_cos_comb.squeeze()[0:]]
results['Distances'] = distances_cos_comb.squeeze()[0:]
json.loads(results.to_json(orient='records'))

[array([[5.6021978, 0.       , 0.       , 1.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 1.       , 1.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.     

<ipython-input-212-b316fddf9459>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['Distances'] = distances_cos_comb.squeeze()[0:]


[{'manga_id': 21,
  'title': 'Death Note',
  'type': 'manga',
  'score': 8.7,
  'status': 'finished',
  'volumes': 12.0,
  'main_picture': 'https://cdn.myanimelist.net/images/manga/1/258245l.jpg',
  'url': 'https://myanimelist.net/manga/21/Death_Note',
  'chapters': 108.0,
  'start_date': '2003-12-01',
  'genres': ['Supernatural', 'Suspense'],
  'themes': ['Psychological'],
  'demographics': ['Shounen'],
  'sfw': True,
  'authors': [{'first_name': 'Takeshi',
    'id': 1888,
    'last_name': 'Obata',
    'role': 'Art'},
   {'first_name': 'Tsugumi',
    'id': 2111,
    'last_name': 'Ohba',
    'role': 'Story'}],
  'title_english': 'Death Note',
  'title_japanese': 'DEATH NOTE',
  'authors_id': '0',
  'text_input': ['Psychological', 'Supernatural', 'Suspense', 'Shounen'],
  'Distances': 0.0},
 {'manga_id': 23900,
  'title': 'Umineko no Naku Koro ni Chiru - Episode 7: Requiem of the Golden Witch',
  'type': 'manga',
  'score': 8.55,
  'status': 'finished',
  'volumes': 9.0,
  'main_picture

In [ ]:
import json
mangac=pd.read_parquet('animovix-models and files/manga.parquet')
pipeline = joblib.load('animovix-models and files/preprocess_pipeline_manga.sav')
K_model_cos=joblib.load('animovix-models and files/K_model_manga.sav')

def recommend(input="demon slayer"): # Input will be separated by '|' and then mid points of the
    input = (input.lower()).split('|')              # corresponding vector representations will be found coresponding to
                                                    # which Final Recommendations will be given as output.
    samples=[]
    for name in input:
        samples.append((pipeline.transform(mangac[mangac['title'].apply(lambda x:name ==  x.lower())][:1])).toarray())

    sample_mean = [sum(sub_list) / len(sub_list) for sub_list in zip(*samples)] # finding mean of all nodes and
    distances_cos_comb, indices_cos_comb = K_model_cos.kneighbors(sample_mean)        # treating it as a midpoint
    results = mangac.iloc[indices_cos_comb.squeeze()[0:]]
    results['Distances'] = distances_cos_comb.squeeze()[0:]
    return json.loads(results.to_json(orient='records'))

In [ ]:
# recommend('kimetsu no yaiba')
results_json = recommend('samurai champloo')
# !pip install memory-profiler
from memory_profiler import memory_usage
results_df = pd.json_normalize(results_json)
# results_df['English'].tolist()
results_json
results_df['title'].tolist()

<ipython-input-216-7673aeb5b60d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['Distances'] = distances_cos_comb.squeeze()[0:]


['Samurai Champloo',
 'Esper Mami',
 'Ueki no Housoku Plus',
 'Digimon Adventure: V-Tamer 01',
 'Juhou Kaikin!! Hyde & Closer',
 'Hameln no Violin-hiki',
 'Slayers: Medieval Mayhem',
 'Wanted!',
 'Samurai Deeper Kyou',
 'Im',
 'Dragon Eye',
 "Mashima Hero's",
 'Black Cat',
 'Pocket Monsters: Ruby-Sapphire-hen',
 'Slayers Premium',
 'Full Ahead! Coco',
 'Toriko',
 'Tekken Chinmi Legends',
 'Hana no Keiji: Kumo no Kanata ni',
 'Monster Collection: Majuu Tsukai no Shoujo',
 'Rekka no Honoo',
 'Magico',
 'Digimon Xros Wars',
 'Anagle Mole',
 'Sakamoto Days',
 'Shaman King',
 'Densetsu no Yuusha no Densetsu',
 'Doubutsu no Kuni',
 'Ozanari Dungeon',
 'Magi: Sinbad no Bouken - Prototype',
 'Suashi no Meteorite',
 'Iken Senki Völundio',
 'A-bout!',
 'Heart Gear',
 'Takeru ~ Opera Susanoh Sword of the Devil',
 'Fairy Tail: 100 Years Quest',
 'Shangri-La Frontier: Kusoge Hunter, Kamige ni Idoman to su',
 'Lucky Star Pocket Travelers',
 'Shin Tekken Chinmi',
 'Mahoujin Guruguru',
 'Ginga Densets